# Исследование данных Garpix на предмет возможности использования классических ML алгоритмов, для решения поставленной задачи. 

In [1]:
import pandas as pd
from classes import GarpixZipReader
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# Загрузка данных, полученных от системы GLS
data_reader = GarpixZipReader('../../data/garpix_data.zip')
raw_data = data_reader.read_zip()

In [14]:
# Первичная очистка данных
full_boxes_list = []  # Полный список всех коробок, без учета грузового пространства
calculation_data = []  # Данные расчетов (Учитываются только коробки, поместившиеся в грузовое отделение) 

for raw_item in raw_data:
    calculation = {}
    for box_item in raw_item['groups'][0]['cargoes']:
        box = {'width': box_item['width'],
               'height': box_item['height'],
               'length': box_item['length'],
               'mass': box_item['mass'],
               'stacking': box_item['stacking'],
               'turnover': box_item['turnover'],
               'is_rotate_y': box_item['is_rotate_y'],
               'stacking_limit': box_item['stacking_limit']
               }
        full_boxes_list.append(box)

    calculation_data.append(calculation)

print(len(full_boxes_list))

204133
